#`5440 Quant. ML/DatSci | SPRING 2021 | ASSIGNMENT 4 | UNI: CHB2132 `#

---

##**Problem 1 (10 points)**

**Part (a) [5 points]**

---

*Write a computer program that implements the Kalman filter*

In [ ]:
# from Columbia Professor William Ritter's lecture notes

In [ ]:
import importlib
importlib.import_module('price_target_data')

from price_target_data import dataList

In [ ]:
import matplotlib.pyplot as plt

plt.plot(dataList())
plt.ylabel('Price Targets')
plt.show()

In [ ]:
import numpy as np
import math
from math import log
from math import pi

In [ ]:
class error(Exception):
  """General class for throwing exceptions"""
  pass

In [ ]:
class gaussian:
  mean = None
  cov = None
  covdet = math.nan

  def __init__(self, mu, Omega):
    self.mean = mu
    self.cov = Omega

  def getcovdet(self):
    if(math.isnan(self.covdet)):
      self.covdet = np.linalg.det(cov)
    return self.covdet

  def logdens(self, x):
    d = mean.size
    xmm = np.subtract(mean)
    return (-d / 2.0) * log(2 * pi) - 0.5 * log(getcovdet()) - 0.5 * np.dot(xmm, getcovinv() @ xmm)

In [ ]:
class gaussian_indexed:
  g = None #Gaussian
  k = None

  def __init__(self, gauss, i):
    self.g = gauss
    self.k = i
  
  def index(self):
    return self.k

  def mean(self):
    return self.g.mean

  def covs(self):
    return self.g.cov

In [ ]:
class vect_indexed:
  vector = None
  k = None

  def __init__(self, vec, i):
    self.vector = vec
    self.k = i

  def index(self):
    return self.k
  
  def __str__(self):
    return str(self.vector) + " at index " + str(self.k)

In [ ]:
#observation
class observation(vect_indexed):
  pass

In [ ]:
#state prediction density
class sp_dens(gaussian_indexed):
  pass

In [ ]:
#obs prediction density
class op_dens(gaussian_indexed):
  pass

In [ ]:
#filter density
class filter_dens(gaussian_indexed):
  pass

In [ ]:
#kalman triple
class kalman_triple:
  spd = None #state prediction density
  opd = None #observation prediction density
  fd = None #filter density
  y = None #observation
  k = None #integer

  def __init__(self, spd1, opd1, fd1, y1):
    self.spd = spd1
    self.opd = opd1
    self.fd = fd1
    self.y = y1
    self.k = spd1.index()

  def normsqerror(self):
    tmp = np.subtract(self.y.self.opd.mean())
    return np.dot(tmp, tmp)

In [ ]:
class kalman_model:
  A = None
  H = None
  Q = None
  R = None
  At = None
  Ht = None
  prior = None

  def __init__(self, a1, h1, q1, r1, prior1):
    self.A = a1 
    self.H = h1
    self.Q = q1
    self.R = r1

    self.At = A.T
    self.Ht = H.T

    self.prior = prior1

  def update(self, spd, y): #returns Kalman triple
    if(y.index() != spd.index()):
      raise error("y.index() = " + str(y.index() + "does not equal spd.index() = " + str(spd.index())))

    if(not isinstance(y, obs)):
      raise error("y is a " + type(y).__name__ + " not an observation")

    if(not isinstance(spd, sp_dens)):
      raise error("spd is a " + type(y).__name__ + "not a state prediction density")

    m_k_minus = spd.mean()
    v = np.subtract(y.vector, self.H @ m_k_minus)
    
    S = self.computeS(spd)
    
    Sinv = np.linalg.inv(S)
    K = spd.cov() @ (self.Ht @ Sinv)

    p = np.subtract(spd.cov(), K @ (S @K.T))
    f = filter_dens(gaussian(m_k, P), y.index())

    opd = op_dens(gaussian(self.H @ m_k_minus, S), y.index())
    return kalman_triple(spd, opd, f, y)

  def computeS(self, spd): 
    return np.add(self.H @ (spd.cov() @ self.Ht), self.R)

  # f needs to be filter density
  def prediction(self, f): #return state prediction density
    if(not isinstance(f, filter_dens)):
      raise error("f is a " + type(f).__name__ + "not a filter density")

    m_minus = self.A @ f.mean()
    X = np.add(self.A @ (f.cov() @ self.At), self.Q)
    return sp_dens(gaussian(m_minus, X), f.index() + 1)
  
  def filter(self, ys):
    # ys needs to be list of observation/obs
    if(not isinstance(ys[0], observation)):
      raise error("ys[0] is a " + type(ys[0]).__name__ + "not an obsercation")

    result = []
    prev = filter_dens(self.prior, 0) #use index 0 for prior

    for y in ys:
      pred = self.prediction(prev)
      triple = self.update(pred, y)
      result.append(triple)
      prev = triple.fd

    return result

### Note ###
---
*We want to consider the vector of parameters and create from this our desired Kalman model.*

*A serves as our transition matrix and H as our observation matrix. We will implement the model based on the function:* y = x_1 + noise

*As well as the matrix equations invoking alpha, Beta, and gamma as our parameters to be fit.*

In [ ]:
#return kalman model
def model_k(params, prior):
  h = np.array([[1.0, 0.0]])
  r = np.array([params[0]])
  q = np.diag([params[1], params[2]])
  a = np.array([[params[3], params[4]], [0.0, params[5]]])
  return kalman_model(a, h, q, r, prior)

In [ ]:
# testing out kalman model with artificial parameters

common_prior = gaussian(np.array([dataList()[1], 0.0]), np.eye(2))

params1 = [0.02938409328783209, #r
           4.290127348917587E-4, 3.110938894283071, #q
           0.920746815344842345, #alpha
           0.2987321987332179, #beta
           2.01283409178758217598128] #gamma
  
kf1 = model_k(params1, common_prior)

In [ ]:
def obs_list(L):
  ys = []
  j = 0

  for obs in L:
    j = j + 1
    ys.append(observation()